In [23]:
import pandas as pd
import pyarrow
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

# Cargar variables de entorno
load_dotenv()

try:
    # Conexión a SalesSystem (Facturación)
    salessystem_url = f"{os.getenv('DB_SALESSYSTEM_DIALECT')}://{os.getenv('DB_SALESSYSTEM_USER')}:{os.getenv('DB_SALESSYSTEM_PASSWORD')}@{os.getenv('DB_SALESSYSTEM_HOST')}:{os.getenv('DB_SALESSYSTEM_PORT')}/{os.getenv('DB_SALESSYSTEM_NAME')}"
    salessystem = create_engine(salessystem_url)
    
    # Conexión a Warehouse (Contabilidad)
    warehouse_url = f"{os.getenv('DB_WAREHOUSE_DIALECT')}://{os.getenv('DB_WAREHOUSE_USER')}:{os.getenv('DB_WAREHOUSE_PASSWORD')}@{os.getenv('DB_WAREHOUSE_HOST')}:{os.getenv('DB_WAREHOUSE_PORT')}/{os.getenv('DB_WAREHOUSE_NAME')}"
    warehouse = create_engine(warehouse_url)
    
    # Validar las conexiones
    with salessystem.connect() as conn_sales:
        pass
    with warehouse.connect() as conn_warehouse:
        pass
        
except Exception as e:
    print(f"Error al conectar a las bases de datos: {str(e)}")
    raise

In [24]:
# VISTA FACTURAS_NOANULADAS FILTRADAS POR PERIODO '202406'
query = ("SELECT * FROM public.facturas_noanuladas_ventas WHERE periodo_tributario = 202510")

# TABLA FACTURAS DE SALES SYSTEM
facturas = pd.read_sql(query, con=warehouse, parse_dates=['fecha_emision','fecha_vencimiento'], dtype_backend="pyarrow")

# TABLA PROVEEDORES DE SALES SYSTEM
proveedores = pd.read_sql("SELECT tipo_proveedor, numero_documento, alias FROM proveedores", con=salessystem, dtype_backend="pyarrow")

# TABLA CLIENTES DE SALES SYSTEM
clientes = pd.read_sql("SELECT * FROM customers", con=salessystem, dtype_backend="pyarrow")

**FACTURAS AGRUPADAS POR ENTIDAD, TOTAL Y CANTIDAD DE COMPROBANTES**

In [25]:
# FACTURAS AGRUPADAS POR PROVEEDOR(ruc), SUMA IGV, SUMA VALOR, SUMA IGV Y CONTEO
result = facturas.groupby('ruc').agg(
    columna1_sum=('valor', 'sum'),
    columna2_sum=('igv', 'sum'),
    count=('ruc', 'size')
).reset_index()
# TOTAL = SUMA IGV + SUMA SUMA VALOR
result['total']=result['columna1_sum']+result['columna2_sum']
# ELIMINAR COLUMNAS SUMA IGV Y SUMA VALOR
result.drop(['columna1_sum', 'columna2_sum'], axis=1, inplace=True)

In [26]:
# FILTRAR SOLO PROVEEDORES TIPO 1 Y 2
proveedores_tipo1y2 = proveedores[proveedores['tipo_proveedor'] < '3']
# FILTRAR SOLO PROVEEDORES TIPO 3
proveedores_tipo3 = proveedores[proveedores['tipo_proveedor'] == '3']
# FILTRAR SOLO CLIENTES INTERNOS REGISTRADOS
clientes_internos = clientes[clientes['observaciones'].str.contains('INTERNO', case=False, na=False)]
# FILTRAR SOLO CLIENTES EXTERNOS REGISTRADOS
clientes_externos = clientes[~clientes['observaciones'].str.contains('INTERNO|PROVEEDOR', case=False, na=False)]

In [27]:
# FILTRAR FACTURAS EMITIDAS A PROVEEDORES TIPO 1 Y 2
filtro1_proveedores = facturas[facturas['numero_documento'].isin(proveedores_tipo1y2['numero_documento'].astype(str))]

In [28]:
# FILTRAR FACTURAS EMITIDAS A PROVEEDORES TIPO 3
filtro2_proveedores = facturas[facturas['numero_documento'].isin(proveedores_tipo3['numero_documento'].astype(str))]

In [29]:
# FILTRAR FACTURAS EMITIDAS A CLIENTES INTERNOS REGISTRADOS
filtro_clientesinternos = facturas[facturas['numero_documento'].isin(clientes_internos['ruc'].astype(str))]

In [30]:
# FILTRAR FACTURAS EMITIDAS A OTROS CLIENTES O A CLIENTES NO REGISTRADOS
filtro_otrosclientes = facturas[~facturas['numero_documento'].isin(clientes['ruc'].astype(str))]
filtro_otrosclientes['numero_documento'].drop_duplicates()

Series([], Name: numero_documento, dtype: string[pyarrow])

In [31]:
# FILTRAR FACTURAS EMITIDAS A CLIENTES EXTERNOS
filtro_clientesexternos = facturas[facturas['numero_documento'].isin(clientes_externos['ruc'].astype(str))]

In [32]:
print('Cantidad de facturas emitidas por facturacion: '+str(len(facturas['cui'])))
print('Cantidad de facturas emitidas a proveedores tipo 1 y 2: '+str(len(filtro1_proveedores['cui'])))
print('Cantidad de facturas emitidas a proveedores tipo 3: '+str(len(filtro2_proveedores['cui'])))
print('Cantidad de facturas emitidas a clientes internos: '+str(len(filtro_clientesinternos['cui'])))
print('Cantidad de facturas emitidas a clientes externos recurrentes: '+str(len(filtro_clientesexternos['cui'])))
print('Cantidad de facturas emitidas a nuevos clientes: '+str(len(filtro_otrosclientes['cui'])))

Cantidad de facturas emitidas por facturacion: 424
Cantidad de facturas emitidas a proveedores tipo 1 y 2: 27
Cantidad de facturas emitidas a proveedores tipo 3: 0
Cantidad de facturas emitidas a clientes internos: 18
Cantidad de facturas emitidas a clientes externos recurrentes: 406
Cantidad de facturas emitidas a nuevos clientes: 0


In [33]:
# FILTRO FACTURAS CON GUIA
filtro_facturaconguia = facturas[facturas['tipo_documento_referencia'] == 9]
# FILTRO FACTURAS SIN GUIA
filtro_facturasinguia = facturas[facturas['tipo_documento_referencia'] == 1]

In [34]:
print('Cantidad de facturas con guia asociada: '+str(len(filtro_facturaconguia['cui'])))
print('Cantidad de facturas sin guia: '+str(len(filtro_facturasinguia['cui'])))

Cantidad de facturas con guia asociada: 411
Cantidad de facturas sin guia: 13


In [35]:
# TABLA PEDIDOS ORDENADO POR ADQUIRIENTE/CLIENTE FILTRADO POR PERIODO '202406'
pedidos = pd.read_sql("SELECT * FROM pedidos WHERE periodo = 202510",salessystem, parse_dates=['fecha_pedido'], dtype_backend="pyarrow").sort_values(by='adquiriente', ascending=True)
# PEDIDOS FILTRADOS POR PEDIDOS ENTREGADOS Y/O EMITIDOS
pedidos_preparados = pedidos[pedidos['estado'].str.contains('ENTREGADO|EMITIDO', case=False, na=False)]

In [36]:
# FACTURAS FILTRADAS POR PEDIDOS REGISTRADOS COMO EMITIDO/ENTREGADO
facturas_pedidos_preparados=facturas[facturas['numero_documento'].isin(pedidos_preparados['adquiriente'].unique().astype(str))]
# FACTURAS FILTADAS POR PEDIDOS REGISTRADOS AGRUPADAS POR ADQUIRIENTE(numero_documento)
pedidos_verificados = facturas_pedidos_preparados.groupby('numero_documento').agg(
    columna1_sum=('valor', 'sum'),
    columna2_sum=('igv', 'sum'),
    count=('ruc', 'size')
).reset_index()
pedidos_verificados['total']=pedidos_verificados['columna1_sum']+pedidos_verificados['columna2_sum']
pedidos_verificados.drop(['columna1_sum', 'columna2_sum'], axis=1, inplace=True)

In [37]:
pedidos_verificados

,numero_documento,count,total


In [38]:
# POR CADA ADQUIRIENTE DEL DATAFRAME pedidos_preparados
for adquiriente in pedidos_preparados['adquiriente'].unique():
    # FILTRAR PEDIDOS POR ADQUIRIENTE
    pedido = pedidos_preparados[pedidos_preparados['adquiriente'] == adquiriente]
    # FACTURAS EMITIDAS PARA LOS PEDIDOS PREPARADOS FILTRADAS POR ADQUIRIENTE (numero_documento)
    total_facturas_adquiriente = pedidos_verificados[pedidos_verificados['numero_documento'] == str(adquiriente)]
    
    # SUMAR TOTAL POR ADQUIRIENTE
    total_por_adquiriente = total_facturas_adquiriente['total'].sum()
    
    # SI EL ADQUIRIENTE TIENE MAS DE 1 PEDIDO EN EL PERIODO
    if len(pedido) > 1:
        print('Varios pedidos de un adquiriente')
    # SI SOLO TIENE 1 PEDIDO    
    else: 
        # OBTENER EL CODIGO DE PEDIDO
        pedido1 = pedido['cod_pedido'].values[0]
        # OBTENER EL IMPORTE TOTAL DEL PEDIDO
        importe = pedido['importe_total'].values[0]
        
        print(f"Pedido codigo {pedido1} por {importe} emitido por {total_por_adquiriente}")

In [39]:
# PEDIDOS EMITIDOS POR FACTURACION AGRUPADOS Y ORDENADOS POR PROVEEDOR(RUC) FILTRADOS POR PERIODO
emitidos_facturacion = pd.read_sql("SELECT ruc,count(distinct concat(cod_pedido,cuo)), SUM(round(cantidad * precio_unit *1.18,2)) AS total FROM facturas WHERE CAST(DATE_FORMAT(emision, '%%Y%%m') AS UNSIGNED) = 202510 GROUP BY ruc ORDER BY ruc", con=salessystem)

In [40]:
emitidos_facturacion

,ruc,"count(distinct concat(cod_pedido,cuo))",total
0,10067597937,6,10000.06
1,10093677469,5,20036.40
2,10105880818,3,1124.13
3,10297073058,4,2001.87
4,10415173101,15,25900.94
5,10470078923,6,6001.42
6,10719796007,6,10000.98
7,10726501306,13,96785.96
8,10779181575,8,5000.37
9,15609947851,3,4001.71


In [41]:
#FACTURAS QUE NO SON A PROVEEDORES
facturas_cliente=facturas[~facturas['numero_documento'].isin(proveedores['numero_documento'].astype(str))]

In [42]:
facturas_cliente_conguia=facturas_cliente[
    (facturas_cliente['tipo_documento_referencia'] == 9) &
    (~facturas_cliente['numero_documento_referencia'].str.startswith('0001'))
]
facturas_cliente_conguia.loc[:, 'numero_documento_referencia'] = facturas_cliente_conguia['numero_documento_referencia'].apply(lambda x: int(x[6:-1]))

In [43]:
facturas_cliente_conguia.sort_values(by=['ruc','numero_documento_referencia'], ascending=True)

,id,ruc,subdiario,periodo_tributario,tipo_operacion,tipo_comprobante,fecha_emision,fecha_vencimiento,numero_serie,numero_correlativo,...,tipo_comprobante_modificado,numero_serie_modificado,numero_correlativo_modificado,glosa,cui,observaciones,cuenta_contable,igv,tipo_documento_referencia,numero_documento_referencia
141,14555,10406916087,5,202510,1,1,2025-10-24,NaT,E001,207,...,<NA>,<NA>,<NA>,<NA>,526c4ceff701E001207,PARSER,<NA>,266.76,9,51
142,14556,10406916087,5,202510,1,1,2025-10-24,NaT,E001,208,...,<NA>,<NA>,<NA>,<NA>,526c4ceff701E001208,PARSER,<NA>,304.38,9,52
143,14557,10406916087,5,202510,1,1,2025-10-24,NaT,E001,209,...,<NA>,<NA>,<NA>,<NA>,526c4ceff701E001209,PARSER,<NA>,297.0,9,53
144,14558,10406916087,5,202510,1,1,2025-10-24,NaT,E001,210,...,<NA>,<NA>,<NA>,<NA>,526c4ceff701E001210,PARSER,<NA>,304.38,9,54
145,14559,10406916087,5,202510,1,1,2025-10-24,NaT,E001,211,...,<NA>,<NA>,<NA>,<NA>,526c4ceff701E001211,PARSER,<NA>,50.4,9,55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,14630,20614580951,5,202510,1,1,2025-10-31,NaT,E001,286,...,<NA>,<NA>,<NA>,<NA>,54ccb98ad701E001286,PARSER,<NA>,168.35,9,282
218,14633,20614580951,5,202510,1,1,2025-10-30,NaT,E001,287,...,<NA>,<NA>,<NA>,<NA>,54ccb98ad701E001287,PARSER,<NA>,283.5,9,283
221,14636,20614580951,5,202510,1,1,2025-10-30,NaT,E001,288,...,<NA>,<NA>,<NA>,<NA>,54ccb98ad701E001288,PARSER,<NA>,187.2,9,284
224,14639,20614580951,5,202510,1,1,2025-10-31,NaT,E001,289,...,<NA>,<NA>,<NA>,<NA>,54ccb98ad701E001289,PARSER,<NA>,324.38,9,285
